In [1]:
from datetime import datetime, timedelta
from siphon.catalog import TDSCatalog
from siphon.ncss import NCSS
import numpy as np
import metpy.calc as mcalc
from metpy.units import units, concatenate
from metpy.plots import SkewT

Just some helper code to make things easier. `metpy_units_handler` plugins into siphon to automatically add units to variables. `post_process_data` is used to clean up some oddities from the NCSS point feature collection.

In [2]:
units.define('degrees_north = 1 degree')
units.define('degrees_east = 1 degree')
unit_remap = dict(inches='inHg', Celsius='celsius')
def metpy_units_handler(vals, unit):
    arr = np.array(vals)
    if unit:
        unit = unit_remap.get(unit, unit)
        arr = arr * units(unit)
    return arr

# Fix dates and sorting
def sort_list(list1, list2):
    return [l1 for (l1, l2) in sorted(zip(list1, list2), key=lambda i: i[1])]

def post_process_data(data):
    data['time'] = [datetime.strptime(d.decode('ascii'), '%Y-%m-%d %H:%M:%SZ') for d in data['time']]
    ret = dict()
    for key,val in data.items():
        try:
            val = units.Quantity(sort_list(val.magnitude.tolist(), data['time']), val.units)
        except AttributeError:
            val = sort_list(val, data['time'])
        ret[key] = val
    return ret

# METAR Meteogram

First we need to grab the catalog for the METAR feature collection data from http://thredds.ucar.edu/thredds/catalog.html

Set up NCSS access to the dataset

What variables do we have available? (Hint: try a `variables` attribute.)

Create a query for the last 7 days of data for a specific lon/lat point. We should ask for: air temperature, dewpoint temperature, wind speed, and wind direction.

Get the data

## Heat Index

First, we need relative humidity:
    $$RH = e / e_s$$

Calculate heat index:

Plot the temperature, dewpoint, and heat index. Bonus points to also plot wind speed and direction.

#Sounding

First grab the catalog for the Best dataset from the GSD HRRR from http://thredds.ucar.edu/thredds/catalog.html

Set up NCSS access to the dataset

What variables do we have?

Set up a query for the most recent set of data from a point. We should request temperature, dewpoint, and U and V.

Get the data

Plot a sounding of the data

Also calculate the parcel profile and add that to the plot

Let's also plot the location of the LCL and the 0 isotherm as well: